# 所有策略使用範例

本 Notebook 展示 MediaOverload 專案中所有生成策略的使用方法。

## 支援的策略

1. **Text2ImageStrategy** - 文生圖
2. **Image2ImageStrategy** - 圖生圖
3. **Text2Image2ImageStrategy** - 文生圖 → 圖生圖
4. **Text2VideoStrategy** - 文生影片
5. **Text2Image2VideoStrategy** - 文生圖 → 圖生影片
6. **Text2LongVideoStrategy** - 文生長影片（尾幀驅動）
7. **Text2LongVideoFirstFrameStrategy** - 文生長影片（首幀驅動）
8. **StickerPackStrategy** - 貼圖包生成

## 使用模式

每個策略都支援兩種使用模式：
- **自定義 Prompt**：直接指定關鍵詞或描述
- **從資料庫獲取 News**：自動從資料庫獲取新聞作為生成素材

## 批量生成

所有策略都支援批量生成，可指定生成數量（如 30 張、50 張等）。


## 環境配置

### ComfyUI 連接設定

在 Windows 環境中運行時，需要設定環境變數：


In [1]:
import sys
import os
from pathlib import Path
from datetime import datetime

# 確保可以導入專案模組
project_root = Path.cwd().parent if Path.cwd().name == 'examples' else Path.cwd()
sys.path.insert(0, str(project_root))

# 設定 ComfyUI 連接環境變數（Windows 環境使用 localhost）
os.environ['COMFYUI_HOST'] = '127.0.0.1'
os.environ['COMFYUI_PORT'] = '8188'

# 導入必要模組
from examples.quick_draw.helpers import FlexibleGenerator, ConfigBuilder
from examples.simple_content_service import SimpleContentGenerationService
from lib.repositories.news_repository import NewsRepository
from lib.repositories.character_repository import CharacterRepository
from lib.database import db_pool
from lib.media_auto.models.vision.vision_manager import VisionManagerBuilder
from dotenv import load_dotenv
from IPython.display import Image, display
import glob

# 載入環境變數
env_path = project_root / 'media_overload.env'
load_dotenv(env_path)

# 初始化資料庫連接
db_pool.initialize('mysql',
                  host=os.environ['mysql_host'],
                  port=int(os.environ['mysql_port']),
                  user=os.environ['mysql_user'],
                  password=os.environ['mysql_password'],
                  db_name=os.environ['mysql_db_name'])

mysql_conn = db_pool.get_connection('mysql')

# 初始化服務
character_repository = CharacterRepository(mysql_conn)
news_repository = NewsRepository(mysql_conn)
vision_manager = VisionManagerBuilder() \
    .with_vision_model('openrouter') \
    .with_text_model('openrouter') \
    .with_random_models(True) \
    .build()

content_service = SimpleContentGenerationService(
    character_repository=character_repository,
    vision_manager=vision_manager
)

generator = FlexibleGenerator(verbose=True)

print("✓ 環境初始化完成")
print(f"✓ ComfyUI 連接設定: {os.environ['COMFYUI_HOST']}:{os.environ['COMFYUI_PORT']}")

2025-12-16 21:56:21,077 - mediaoverload - INFO - 隨機選擇的 Vision 模型: qwen/qwen2.5-vl-32b-instruct:free
2025-12-16 21:56:21,080 - mediaoverload - INFO - 隨機選擇的 Text 模型: x-ai/grok-4.1-fast:free
2025-12-16 21:56:21,086 - mediaoverload - INFO - 隨機選擇的 Vision 模型: qwen/qwen2.5-vl-32b-instruct:free
2025-12-16 21:56:21,087 - mediaoverload - INFO - 隨機選擇的 Text 模型: deepseek/deepseek-chat-v3.1:free


正在載入環境變數: C:\Users\jaesm14774\Desktop\self_project\mediaoverload\media_overload.env
環境變數載入成功
✓ 服務初始化完成
✓ 環境初始化完成
✓ ComfyUI 連接設定: 127.0.0.1:8188


## 輔助函數

定義用於獲取新聞和批量生成的輔助函數：


In [3]:
def get_news_from_db(days_back=1):
    """從資料庫獲取新聞
    
    Args:
        days_back: 回溯天數，預設 1 天
        
    Returns:
        包含 title 和 keyword 的字典，如果沒有則返回 None
    """
    date_filter = (datetime.now() - timedelta(days=days_back)).strftime('%Y-%m-%d')
    news = news_repository.get_random_news(date_filter)
    
    if news:
        print(f"📰 獲取到新聞:")
        print(f"   標題: {news['title']}")
        print(f"   關鍵詞: {news['keyword']}")
        return news
    else:
        print("⚠️ 未找到符合條件的新聞")
        return None

def batch_generate_by_count(strategy_type, num_total, 
                           use_news=False, 
                           custom_keywords=None,
                           character="kirby",
                           system_prompt="stable_diffusion_prompt",
                           **kwargs):
    """批量生成指定數量的媒體
    
    Args:
        strategy_type: 策略類型 ('text2image', 'text2video', 等)
        num_total: 總共要生成的數量
        use_news: 是否使用新聞（True）或自定義關鍵詞（False）
        custom_keywords: 自定義關鍵詞（當 use_news=False 時使用）
        character: 角色名稱
        system_prompt: 系統提示詞
        **kwargs: 其他策略特定參數
        
    Returns:
        生成結果列表
    """
    results = []
        
    # 計算需要生成幾輪（每輪生成數量由策略決定）
    if strategy_type in ['text2image', 'text2img']:
        images_per_batch = kwargs.get('num_images', 4)
        num_batches = (num_total + images_per_batch - 1) // images_per_batch
        items_per_batch = images_per_batch
    elif strategy_type in ['text2video', 't2v']:
        videos_per_batch = kwargs.get('num_videos', 2)
        num_batches = (num_total + videos_per_batch - 1) // videos_per_batch
        items_per_batch = videos_per_batch
    elif strategy_type in ['text2image2image', 'image2image']:
        images_per_batch = kwargs.get('num_images', 4)
        num_batches = (num_total + images_per_batch - 1) // images_per_batch
        items_per_batch = images_per_batch
    elif strategy_type == 'text2image2video':
        videos_per_batch = kwargs.get('num_videos_per_image', 1)
        # 這裡假設 num_total 是要生成的影片總數
        num_batches = (num_total + videos_per_batch - 1) // videos_per_batch
        items_per_batch = videos_per_batch
    elif strategy_type in ['text2longvideo', 'text2longvideo_firstframe']:
        # 長影片每次生成 1 個完整影片
        num_batches = num_total
        items_per_batch = 1
    elif strategy_type == 'sticker_pack':
        # 貼圖包每次生成多個表情
        stickers_per_batch = kwargs.get('expressions_count', 8)
        num_batches = (num_total + stickers_per_batch - 1) // stickers_per_batch
        items_per_batch = stickers_per_batch
    else:
        # 其他策略預設每批 1 個
        num_batches = num_total
        items_per_batch = 1
    
    print(f"\n📦 開始批量生成")
    print(f"   策略: {strategy_type}")
    print(f"   目標數量: {num_total}")
    print(f"   批次數: {num_batches}")
    print(f"   模式: {'從資料庫獲取新聞' if use_news else '自定義關鍵詞'}")
    print("="*60)
    
    for i in range(num_batches):
        print(f"\n[{i+1}/{num_batches}] 處理中...")
        
        # 獲取關鍵詞
        if use_news:
            news = get_news_from_db()
            if not news:
                print("⚠️ 無法獲取新聞，跳過此批次")
                continue
            keywords = news['keyword'] or news['title']
        else:
            keywords = custom_keywords or "peaceful scene, beautiful landscape"
        
        # 根據策略類型生成
        try:
            if strategy_type in ['text2image', 'text2img']:
                result = generator.generate_images(
                    keywords=keywords,
                    system_prompt=system_prompt,
                    character=character,
                    num_images=kwargs.get('num_images', 4),
                    output_subdir=f'batch_{strategy_type}_{i+1}',
                    **{k: v for k, v in kwargs.items() if k != 'num_images'}
                )
            elif strategy_type in ['text2video', 't2v']:
                result = generator.generate_videos(
                    keywords=keywords,
                    system_prompt=system_prompt,
                    character=character,
                    num_videos=kwargs.get('num_videos', 2),
                    output_subdir=f'batch_{strategy_type}_{i+1}',
                    **{k: v for k, v in kwargs.items() if k != 'num_videos'}
                )
            else:
                # 其他策略需要使用 ConfigBuilder
                config = build_config_for_strategy(
                    strategy_type, keywords, character, 
                    system_prompt, **kwargs
                )
                result = content_service.generate_content(config)
            
            results.append({
                'batch': i+1,
                'keywords': keywords,
                'result': result
            })
            
            print(f"✅ 批次 {i+1} 完成，生成 {len(result.get('media_files', []))} 個檔案")
            
        except Exception as e:
            print(f"❌ 批次 {i+1} 失敗: {str(e)}")
            continue
    
    print("\n" + "="*60)
    total_files = sum(len(r['result'].get('media_files', [])) for r in results)
    print(f"✅ 批量生成完成！總共生成: {total_files} 個檔案")
    
    return results

def build_config_for_strategy(strategy_type, keywords, character,
                              system_prompt, **kwargs):
    """為特定策略建立配置"""
    # 假設 project_root 是一個已經定義好的 pathlib.Path 物件
    # (在實際運行環境中，project_root 必須先被定義)
    from pathlib import Path
    
    # 為了讓此片段獨立，假設 project_root 是當前目錄的父目錄 (請根據實際情況調整)
    # 警告：您需要確保 project_root 和 ConfigBuilder 在執行環境中是可用的。
    
    # 模擬 project_root 的定義 (請根據您的實際程式碼調整)
    try:
        project_root # 檢查是否已定義
    except NameError:
        project_root = Path(__file__).resolve().parent.parent

    workflow_folder = project_root / 'configs' / 'workflow'
    output_dir = project_root / 'output_media' / f'{strategy_type}_output'
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # 假設 ConfigBuilder 是已定義的類別
    # builder = ConfigBuilder() # 替換為您的實際 ConfigBuilder 實例化
    
    # 為了示範，這裡使用一個簡單的字典來模擬 builder 的行為
    class MockConfigBuilder:
        def __init__(self):
            self.config = {}
        def with_character(self, character): self.config['character'] = character; return self
        def with_prompt(self, keywords): self.config['keywords'] = keywords; return self
        def with_output_dir(self, output_dir): self.config['output_dir'] = output_dir; return self
        def with_image_system_prompt(self, system_prompt): self.config['system_prompt'] = system_prompt; return self
        def with_generation_type(self, strategy_type): self.config['strategy_type'] = strategy_type; return self
        def with_input_image(self, path): self.config['input_image_path'] = path; return self
        def with_denoise(self, denoise): self.config['denoise'] = denoise; return self
        def with_additional_params(self, **params): self.config.setdefault('additional_params', {}).update(params); return self
        def with_workflow(self, workflow_path): self.config['workflow_path'] = workflow_path; return self
        def build(self): return self.config
    
    builder = MockConfigBuilder()

    builder = builder \
        .with_character(character) \
        .with_prompt(keywords) \
        .with_output_dir(str(output_dir)) \
        .with_image_system_prompt(system_prompt) \
        .with_generation_type(strategy_type)
    
    # 策略特定配置
    if strategy_type == 'image2image':
        builder.with_input_image(kwargs.get('input_image_path', ''))
        builder.with_denoise(kwargs.get('denoise', 0.6))
    elif strategy_type == 'text2image2image':
        builder.with_additional_params(**{
            'strategies': {
                'text2image2image': {
                    'first_stage': {
                        'images_per_description': kwargs.get('num_images', 4),
                        'workflow_path': str(workflow_folder / 'nova-anime-xl.json')
                    },
                    'second_stage': {
                        'images_per_input': kwargs.get('images_per_input', 2),
                        'workflow_path': str(workflow_folder / 'image_to_image.json')
                    }
                }
            }
        })
    elif strategy_type == 'text2image2video':
        builder.with_additional_params(**{
            'strategies': {
                'text2image2video': {
                    'first_stage': {
                        'images_per_description': kwargs.get('num_images', 1),
                        't2i_workflow_path': str(workflow_folder / 'nova-anime-xl.json')
                    },
                    'video': {
                        'videos_per_image': kwargs.get('num_videos_per_image', 1),
                        'i2v_workflow_path': str(workflow_folder / 'wan2.2_gguf_i2v_audio.json')
                    }
                }
            }
        })
    elif strategy_type in ['text2longvideo', 'text2longvideo_firstframe']:
        skip_candidate = kwargs.get('skip_candidate_stage', True)
        builder.with_additional_params(**{
            'strategies': {
                strategy_type: {
                    'longvideo_config': {
                        'skip_candidate_stage': skip_candidate,
                        'segment_count': kwargs.get('segment_count', 3),
                        'segment_duration': kwargs.get('segment_duration', 5),
                        'use_tts': kwargs.get('use_tts', True),
                        'tts_voice': kwargs.get('tts_voice', 'en-US-AriaNeural'),
                        'fps': kwargs.get('fps', 16)
                    },
                    'first_stage': {
                        'workflow_path': str(workflow_folder / 'nova-anime-xl.json'),
                        'style': kwargs.get('style', ''),
                        'batch_size': 1 if skip_candidate else kwargs.get('batch_size', 3)
                    },
                    'video_generation': {
                        'workflow_path': str(workflow_folder / 'wan2.2_gguf_i2v.json'),
                        'use_noise_seed': False
                    }
                }
            }
        })
        if strategy_type == 'text2longvideo_firstframe':
            # 這裡需要存取 builder 內部結構，這依賴於 ConfigBuilder 的設計
            # 假設 builder.config 是儲存配置的地方 (如果是使用 MockConfigBuilder)
            if hasattr(builder, 'config') and 'additional_params' in builder.config and 'strategies' in builder.config['additional_params']:
                builder.config['additional_params']['strategies'][strategy_type]['frame_transition'] = {
                    'enabled': True,
                    'workflow_path': str(workflow_folder / 'image_to_image.json'),
                    'denoise': 0.55
                }
            # 如果使用真正的 ConfigBuilder，這應該是透過一個方法來實現
            # 例如：builder.with_frame_transition_config(...)
            
    elif strategy_type == 'sticker_pack':
        builder.with_additional_params(**{
            'strategies': {
                'sticker_pack': {
                    'expressions_count': kwargs.get('expressions_count', 10),
                    'animated_config': {
                        'enabled': kwargs.get('animated_enabled', True)
                    }
                }
            }
        })
    
    # 設定工作流
    default_workflow = kwargs.get('workflow', 'nova-anime-xl.json')
    builder.with_workflow(str(workflow_folder / default_workflow))
    
    return builder.build()

from datetime import timedelta

print("✓ 輔助函數定義完成")


✓ 輔助函數定義完成


# 1. Text2ImageStrategy - 文生圖

最基本的策略，從文字描述生成圖片。


## 1.1 自定義 Prompt 範例


In [ ]:
# 使用自定義關鍵詞生成圖片
result = generator.generate_images(
    keywords=["櫻花", "春天", "溫暖的陽光"],
    system_prompt="stable_diffusion_prompt",
    character="kirby",
    num_images=4,
    output_subdir="text2image_custom"
)

print(f"\n✅ 生成完成！共 {len(result['media_files'])} 張圖片")
for i, img_path in enumerate(result['media_files'], 1):
    print(f"   圖片 {i}: {img_path}")


## 1.2 從資料庫獲取 News 範例


In [ ]:
# 從資料庫獲取新聞並生成圖片
news = get_news_from_db(days_back=7)

if news:
    result = generator.generate_images(
        keywords=news['keyword'] or news['title'],
        system_prompt="sticker_prompt_system_prompt",
        character="kirby",
        num_images=4,
        output_subdir="text2image_news"
    )
    
    print(f"\n✅ 生成完成！共 {len(result['media_files'])} 張圖片")
    for i, img_path in enumerate(result['media_files'], 1):
        print(f"   圖片 {i}: {img_path}")


## 1.3 批量生成範例（生成 X 張圖片）


In [ ]:
# 批量生成 30 張圖片（使用自定義關鍵詞）
results = batch_generate_by_count(
    strategy_type='text2image',
    num_total=30,
    use_news=False,
    custom_keywords="peaceful scene, beautiful landscape",
    character="kirby",
    system_prompt="stable_diffusion_prompt",
    num_images=4  # 每批次生成 4 張
)

# 顯示結果
total_generated = sum(len(r['result'].get('media_files', [])) for r in results)
print(f"\n📊 總共生成: {total_generated} 張圖片")


In [ ]:
# 批量生成 50 張圖片（使用資料庫新聞）
results = batch_generate_by_count(
    strategy_type='text2image',
    num_total=10,
    use_news=True,  # 從資料庫獲取新聞
    character="kirby",
    system_prompt="stable_diffusion_prompt",
    num_images=4
)

# 顯示結果
total_generated = sum(len(r['result'].get('media_files', [])) for r in results)
print(f"\n📊 總共生成: {total_generated} 張圖片")


# 2. Image2ImageStrategy - 圖生圖

基於現有圖片進行風格轉換或修改。


## 2.1 自定義 Prompt 範例

⚠️ 注意：需要先有一張輸入圖片


In [ ]:
# 先找一張現有的圖片作為輸入
input_image = None
for pattern in ['output_media/**/*.png', 'output_media/*.png']:
    images = glob.glob(pattern, recursive=True)
    if images:
        input_image = images[0]
        break

if input_image:
    print(f"📷 使用輸入圖片: {input_image}")
    
    config = build_config_for_strategy(
        strategy_type='image2image',
        keywords="watercolor style, artistic",
        character="kirby",
        system_prompt="stable_diffusion_prompt",
        input_image_path=input_image,
        denoise=0.6,  # 0.5-0.7 之間，值越小越接近原圖
        workflow='image_to_image.json'
    )
    
    result = content_service.generate_content(config)
    print(f"\n✅ 生成完成！共 {len(result['media_files'])} 張圖片")
else:
    print("⚠️ 未找到輸入圖片，請先運行 text2image 生成一些圖片")


## 2.2 從資料庫獲取 News 範例


In [ ]:
# 從資料庫獲取新聞作為提示詞
news = get_news_from_db()

if news and input_image:
    config = build_config_for_strategy(
        strategy_type='image2image',
        keywords=news['keyword'] or news['title'],
        character="kirby",
        system_prompt="stable_diffusion_prompt",
        input_image_path=input_image,
        denoise=0.6,
        workflow='image_to_image.json'
    )
    
    result = content_service.generate_content(config)
    print(f"\n✅ 生成完成！共 {len(result['media_files'])} 張圖片")
else:
    print("⚠️ 需要新聞和輸入圖片才能執行")


# 3. Text2Image2ImageStrategy - 文生圖 → 圖生圖

兩階段生成：先從文字生成圖片，再對圖片進行風格轉換。


## 3.1 自定義 Prompt 範例


In [ ]:
# 文生圖 → 圖生圖
config = build_config_for_strategy(
    strategy_type='text2image2image',
    keywords="sunset, ocean, peaceful",
    character="kirby",
    system_prompt="stable_diffusion_prompt",
    num_images=2,  # 第一階段生成 2 張
    images_per_input=2,  # 第二階段每張生成 2 張
    workflow='nova-anime-xl.json'
)

result = content_service.generate_content(config)
print(f"\n✅ 生成完成！共 {len(result['media_files'])} 張圖片")


## 3.2 從資料庫獲取 News 範例


In [ ]:
# 使用新聞生成
news = get_news_from_db()

if news:
    config = build_config_for_strategy(
        strategy_type='text2image2image',
        keywords=news['keyword'] or news['title'],
        character="kirby",
        system_prompt="stable_diffusion_prompt",
        num_images=2,
        images_per_input=2,
        workflow='nova-anime-xl.json'
    )
    
    result = content_service.generate_content(config)
    print(f"\n✅ 生成完成！共 {len(result['media_files'])} 張圖片")


# 4. Text2VideoStrategy - 文生影片

直接從文字描述生成影片。


## 4.1 自定義 Prompt 範例


In [ ]:
# 生成影片
result = generator.generate_videos(
    keywords=["flying", "stars", "night sky", "peaceful"],
    system_prompt="stable_diffusion_prompt",
    character="kirby",
    num_videos=2,
    output_subdir="text2video_custom"
)

print(f"\n✅ 生成完成！共 {len(result['media_files'])} 個影片")
for i, video_path in enumerate(result['media_files'], 1):
    print(f"   影片 {i}: {video_path}")


## 4.2 從資料庫獲取 News 範例


In [ ]:
# 使用新聞生成影片
news = get_news_from_db()

if news:
    result = generator.generate_videos(
        keywords=news['keyword'] or news['title'],
        system_prompt="stable_diffusion_prompt",
        character="kirby",
        num_videos=2,
        output_subdir="text2video_news"
    )
    
    print(f"\n✅ 生成完成！共 {len(result['media_files'])} 個影片")


## 4.3 批量生成範例（生成 10 個影片）


In [ ]:
# 批量生成 10 個影片
results = batch_generate_by_count(
    strategy_type='text2video',
    num_total=2,
    use_news=True,  # 從資料庫獲取新聞
    character="kirby",
    system_prompt="stable_diffusion_prompt",
    num_videos=2  # 每批次生成 2 個
)

total_generated = sum(len(r['result'].get('media_files', [])) for r in results)
print(f"\n📊 總共生成: {total_generated} 個影片")


# 5. Text2Image2VideoStrategy - 文生圖 → 圖生影片

兩階段生成：先從文字生成圖片，再將圖片轉換為影片。


## 5.1 自定義 Prompt 範例


In [ ]:
# 文生圖 → 圖生影片
result = generator.generate_text2image2video(
    keywords=["cinematic shot", "dramatic lighting", "slow motion"],
    character="kirby",
    style="movie style",
    num_images=1,  # 第一階段生成 1 張圖片
    num_videos_per_image=1,  # 第二階段每張圖片生成 1 個影片
    output_subdir="text2image2video_custom"
)

print(f"\n✅ 生成完成！共 {len(result['media_files'])} 個影片")
for i, video_path in enumerate(result['media_files'], 1):
    print(f"   影片 {i}: {video_path}")


## 5.2 從資料庫獲取 News 範例


In [ ]:
# 使用新聞生成
news = get_news_from_db()

if news:
    result = generator.generate_text2image2video(
        keywords=news['keyword'] or news['title'],
        character="kirby",
        num_images=1,
        num_videos_per_image=1,
        output_subdir="text2image2video_news"
    )
    
    print(f"\n✅ 生成完成！共 {len(result['media_files'])} 個影片")


# 6. Text2LongVideoStrategy - 文生長影片（尾幀驅動）

生成長影片，使用尾幀作為下一段的起始。


## 6.1 自定義 Prompt 範例


In [ ]:
# 文生長影片（尾幀驅動）
config = build_config_for_strategy(
    strategy_type='text2longvideo',
    keywords="adventure story, journey through different landscapes",
    character="kirby",
    system_prompt="stable_diffusion_prompt",
    workflow='nova-anime-xl.json'
)

# 添加長影片特定配置
config.additional_params = config.additional_params or {}
config.additional_params['strategies'] = {
    'text2longvideo': {
        'longvideo_config': {
            'segment_count': 3,  # 生成 5 個段落
            'segment_duration': 3,  # 每個段落 5 秒
            'use_tts': True  # 是否使用 TTS
        },
        'first_stage': {
            'workflow_path': str(project_root / 'configs' / 'workflow' / 'nova-anime-xl.json'),
            'batch_size': 3
        },
        'video_generation': {
            'workflow_path': str(project_root / 'configs' / 'workflow' / 'wan2.2_gguf_i2v.json')
        }
    }
}

result = content_service.generate_content(config)
print(f"\n✅ 生成完成！共 {len(result.get('media_files', []))} 個影片段落")


## 6.2 從資料庫獲取 News 範例


In [ ]:
# 使用新聞生成長影片
news = get_news_from_db()

if news:
    config = build_config_for_strategy(
        strategy_type='text2longvideo',
        keywords=news['keyword'] or news['title'],
        character="kirby",
        system_prompt="stable_diffusion_prompt",
        workflow='nova-anime-xl.json'
    )
    
    config.additional_params = config.additional_params or {}
    config.additional_params['strategies'] = {
        'text2longvideo': {
            'longvideo_config': {
                'segment_count': 3,
                'segment_duration': 5,
                'use_tts': False
            },
            'first_stage': {
                'workflow_path': str(project_root / 'configs' / 'workflow' / 'nova-anime-xl.json'),
                'batch_size': 2
            },
            'video_generation': {
                'workflow_path': str(project_root / 'configs' / 'workflow' / 'wan2.2_gguf_i2v.json')
            }
        }
    }
    
    result = content_service.generate_content(config)
    print(f"\n✅ 生成完成！")


## 6.3 批量生成範例（直接生成模式）

🆕 使用直接生成模式，不保存中間圖片，只輸出最終完整影片（含 TTS）


In [ ]:
# 批量生成長影片（直接模式）
results = batch_generate_by_count(
    strategy_type='text2longvideo',
    num_total=2,  # 生成 2 個長影片
    use_news=True,  # 從資料庫獲取新聞
    character="kirby",
    system_prompt="stable_diffusion_prompt",
    skip_candidate_stage=True,  # 🆕 直接生成模式，不保存中間圖片
    segment_count=3,  # 3 個段落
    segment_duration=5,  # 每段 5 秒
    use_tts=True,  # 使用 TTS 旁白
    workflow='nova-anime-xl.json'
)

total_generated = sum(len(r['result'].get('media_files', [])) for r in results)
print(f"\n📊 總共生成: {total_generated} 個完整影片")
print("\n💡 提示：直接生成模式只會保存最終的完整影片，不會保存中間圖片和段落影片")


# 7. Text2LongVideoFirstFrameStrategy - 文生長影片（首幀驅動）

生成長影片，使用 I2I 轉換首幀以保持風格連貫。


## 7.1 自定義 Prompt 範例


In [ ]:
# 文生長影片（首幀驅動）
config = build_config_for_strategy(
    strategy_type='text2longvideo_firstframe',
    keywords="epic journey, changing scenes, cinematic",
    character="kirby",
    system_prompt="stable_diffusion_prompt",
    workflow='nova-anime-xl.json'
)

# 添加首幀驅動特定配置
config.additional_params = config.additional_params or {}
config.additional_params['strategies'] = {
    'text2longvideo_firstframe': {
        'longvideo_config': {
            'segment_count': 4,
            'segment_duration': 5,
            'use_tts': False
        },
        'first_stage': {
            'workflow_path': str(project_root / 'configs' / 'workflow' / 'nova-anime-xl.json'),
            'batch_size': 2
        },
        'frame_transition': {
            'enabled': True,
            'workflow_path': str(project_root / 'configs' / 'workflow' / 'image_to_image.json'),
            'denoise': 0.55  # 保持風格，允許場景變化
        },
        'video_generation': {
            'workflow_path': str(project_root / 'configs' / 'workflow' / 'wan2.2_gguf_i2v.json')
        }
    }
}

result = content_service.generate_content(config)
print(f"\n✅ 生成完成！")


## 7.2 從資料庫獲取 News 範例


In [ ]:
# 使用新聞生成長影片（首幀驅動）
news = get_news_from_db()

if news:
    config = build_config_for_strategy(
        strategy_type='text2longvideo_firstframe',
        keywords=news['keyword'] or news['title'],
        character="kirby",
        system_prompt="stable_diffusion_prompt",
        workflow='nova-anime-xl.json'
    )
    
    config.additional_params = config.additional_params or {}
    config.additional_params['strategies'] = {
        'text2longvideo_firstframe': {
            'longvideo_config': {
                'segment_count': 3,
                'segment_duration': 5,
                'use_tts': False
            },
            'first_stage': {
                'workflow_path': str(project_root / 'configs' / 'workflow' / 'nova-anime-xl.json'),
                'batch_size': 2
            },
            'frame_transition': {
                'enabled': True,
                'workflow_path': str(project_root / 'configs' / 'workflow' / 'image_to_image.json'),
                'denoise': 0.55
            },
            'video_generation': {
                'workflow_path': str(project_root / 'configs' / 'workflow' / 'wan2.2_gguf_i2v.json')
            }
        }
    }
    
    result = content_service.generate_content(config)
    print(f"\n✅ 生成完成！")


# 8. StickerPackStrategy - 貼圖包生成

生成多個表情貼圖，可選生成動畫 GIF。


## 8.1 自定義 Prompt 範例


In [ ]:
# 生成貼圖包
config = build_config_for_strategy(
    strategy_type='sticker_pack',
    keywords="happy, sad, angry, surprised, sleepy",  # 表情關鍵詞
    character="kirby",
    system_prompt="sticker_prompt_system_prompt",  # 貼圖專用提示詞
    workflow='nova-anime-xl.json',
    expressions_count=10,  # 生成 10 個表情
    animated_enabled=True  # 啟用動畫 GIF
)

result = content_service.generate_content(config)
print(f"\n✅ 生成完成！")
print(f"   靜態貼圖: {len(result.get('media_files', []))} 張")

# 注意：動畫 GIF 需要通過 handle_review_result 選擇後生成


## 8.2 從資料庫獲取 News 範例


In [ ]:
# 使用新聞生成貼圖包
news = get_news_from_db()

if news:
    config = build_config_for_strategy(
        strategy_type='sticker_pack',
        keywords=news['keyword'] or news['title'],
        character="kirby",
        system_prompt="sticker_prompt_system_prompt",
        workflow='nova-anime-xl.json',
        expressions_count=8,
        animated_enabled=True
    )
    
    result = content_service.generate_content(config)
    print(f"\n✅ 生成完成！")


## 8.3 批量生成範例（生成 30 個表情）


In [ ]:
# 批量生成貼圖包
results = []
num_batches = 3  # 生成 3 批次，每批次 10 個表情

for i in range(num_batches):
    print(f"\n[{i+1}/{num_batches}] 生成貼圖包...")
    
    news = get_news_from_db()
    if news:
        config = build_config_for_strategy(
            strategy_type='sticker_pack',
            keywords=news['keyword'] or news['title'],
            character="kirby",
            system_prompt="sticker_prompt_system_prompt",
            workflow='nova-anime-xl.json',
            expressions_count=10,
            animated_enabled=False  # 批量生成時關閉動畫以加快速度
        )
        
        result = content_service.generate_content(config)
        results.append({
            'batch': i+1,
            'result': result
        })
        
        print(f"✅ 批次 {i+1} 完成")

total_stickers = sum(len(r['result'].get('media_files', [])) for r in results)
print(f"\n📊 總共生成: {total_stickers} 個貼圖")


# 總結

本 Notebook 展示了所有策略的使用方法：

1. ✅ **Text2ImageStrategy** - 文生圖
2. ✅ **Image2ImageStrategy** - 圖生圖  
3. ✅ **Text2Image2ImageStrategy** - 文生圖 → 圖生圖
4. ✅ **Text2VideoStrategy** - 文生影片
5. ✅ **Text2Image2VideoStrategy** - 文生圖 → 圖生影片
6. ✅ **Text2LongVideoStrategy** - 文生長影片（尾幀驅動）
7. ✅ **Text2LongVideoFirstFrameStrategy** - 文生長影片（首幀驅動）
8. ✅ **StickerPackStrategy** - 貼圖包生成

每個策略都支援：
- ✅ 自定義 Prompt
- ✅ 從資料庫獲取 News
- ✅ 批量生成（可指定數量）

## 使用提示

1. **批量生成**：使用 `batch_generate_by_count()` 函數可以輕鬆生成指定數量的媒體
2. **新聞獲取**：使用 `get_news_from_db()` 可以從資料庫獲取新聞作為生成素材
3. **策略選擇**：根據需求選擇合適的策略，例如：
   - 簡單圖片生成 → Text2ImageStrategy
   - 風格轉換 → Image2ImageStrategy
   - 高品質影片 → Text2Image2VideoStrategy
   - 長影片 → Text2LongVideoStrategy 或 Text2LongVideoFirstFrameStrategy
   - 表情貼圖 → StickerPackStrategy


In [ ]:
results = batch_generate_by_count(
    strategy_type='text2image',
    num_total=10,
    use_news=True,
    character="kirby",
    num_images=2
)


📦 開始批量生成
   策略: text2image
   目標數量: 10
   批次數: 5
   模式: 從資料庫獲取新聞

[1/5] 處理中...


2025-12-16 21:58:12,406 - examples.simple_content_service - INFO - 開始簡化內容生成流程（跳過分析和文章生成）
2025-12-16 21:58:12,410 - examples.simple_content_service - INFO - 使用策略: text2img
2025-12-16 21:58:12,411 - examples.simple_content_service - INFO - 策略配置載入完成
2025-12-16 21:58:12,412 - examples.simple_content_service - INFO - 開始生成描述
2025-12-16 21:58:12,413 - examples.simple_content_service - INFO - 採用圖片生成策略 : stable_diffusion_prompt


📰 獲取到新聞:
   標題: 台中第3區合法殯葬業者評鑑出爐 11家獲特優
   關鍵詞: 台中市;永續

🎨 開始生成圖片...
🔖 Keywords: 台中市;永續
📝 System Prompt: stable_diffusion_prompt
👤 Character: kirby
📊 數量: 2
📝 使用的 prompt (關鍵詞): 台中市;永續
Using image system prompt key: stable_diffusion_prompt
📝 傳遞給 LLM 的 user_input (關鍵詞): Main character: kirby
台中市;永續


OpenRouter API 請求失敗 (嘗試 1/10): HTTP None - 404 Client Error: Not Found for url: https://openrouter.ai/api/v1/chat/completions。（當前模型: x-ai/grok-4.1-fast:free）等待 4.1 秒後重試...
OpenRouter API 請求失敗 (嘗試 2/10): HTTP None - 404 Client Error: Not Found for url: https://openrouter.ai/api/v1/chat/completions。（當前模型: minimax/minimax-m2:free）等待 6.7 秒後重試...
OpenRouter API 請求失敗 (嘗試 3/10): HTTP None - 404 Client Error: Not Found for url: https://openrouter.ai/api/v1/chat/completions。（當前模型: tngtech/deepseek-r1t2-chimera:free）等待 11.2 秒後重試...
2025-12-16 21:58:55,697 - examples.simple_content_service - INFO - 描述生成完成，共 1 個描述
2025-12-16 21:58:55,697 - examples.simple_content_service - INFO - 描述 1: "Kirby, the cheerful pink puffball, floats joyfully above Taichung City's sun-drenched sustainable urban park, surrounded by miniature wind turbines and vertical gardens growing Taiwanese cherry blossoms, golden hour sunlight casting long shadows across solar-paneled rooftops and bamboo-structured eco-pavilions, vib

Image descriptions : ['"Kirby, the cheerful pink puffball, floats joyfully above Taichung City\'s sun-drenched sustainable urban park, surrounded by miniature wind turbines and vertical gardens growing Taiwanese cherry blossoms, golden hour sunlight casting long shadows across solar-paneled rooftops and bamboo-structured eco-pavilions, vibrant banyan trees framing the skyline of the Taichung Gateway Park development, digital painting with soft cel-shaded textures and iridescent environmental details"']
生成描述花費 : 43.28 秒
工作流已提交，prompt_id: 8674a29f-b734-4fd8-b3a5-618342db00a0
開始等待工作流 8674a29f-b734-4fd8-b3a5-618342db00a0 完成...
  → 佇列中還有 1 個任務
  → 佇列中還有 1 個任務
  → 正在處理節點: 182
  → 正在處理節點: 260:255
  → 正在處理節點: 260:257
  → 正在處理節點: 260:258
  → 正在處理節點: 260:259
  → 正在處理節點: 272:264
  → 正在處理節點: 272:266
  → 正在處理節點: 272:267
  → 正在處理節點: 272:268
  → 正在處理節點: 269:264
  → 正在處理節點: 269:266
  → 正在處理節點: 269:267
  → 正在處理節點: 269:268
✓ 工作流 8674a29f-b734-4fd8-b3a5-618342db00a0 執行完成（耗時 64.74 秒）
工作流 8674a29f-b734-4fd

...

✓ 工作流 553e5b7f-eda1-4b7e-999d-dbb4741f4fc9 執行完成（耗時 60.46 秒）
工作流 553e5b7f-eda1-4b7e-999d-dbb4741f4fc9 執行完成
✅ 生成圖片總耗時: 244.94 秒
✅ 完成！生成了 24 張圖片
📂 保存位置: C:\Users\jaesm14774\Desktop\self_project\mediaoverload\output_media\batch_text2image_2
✅ 批次 2 完成，生成 24 個檔案

[3/5] 處理中...


2025-12-16 22:10:34,437 - examples.simple_content_service - INFO - 開始簡化內容生成流程（跳過分析和文章生成）
2025-12-16 22:10:34,440 - examples.simple_content_service - INFO - 使用策略: text2img
2025-12-16 22:10:34,441 - examples.simple_content_service - INFO - 策略配置載入完成
2025-12-16 22:10:34,442 - examples.simple_content_service - INFO - 開始生成描述
2025-12-16 22:10:34,443 - examples.simple_content_service - INFO - 採用圖片生成策略 : stable_diffusion_prompt


📰 獲取到新聞:
   標題: 撞擊畫面曝！中和女騎士閃違停車遭大貨車輾過「大腿斷肢」
   關鍵詞: 新北市;中和區;車禍;機車;中和斷肢;大腿;卡車;送醫搶救

🎨 開始生成圖片...
🔖 Keywords: 新北市;中和區;車禍;機車;中和斷肢;大腿;卡車;送醫搶救
📝 System Prompt: stable_diffusion_prompt
👤 Character: kirby
📊 數量: 2
📝 使用的 prompt (關鍵詞): 新北市;中和區;車禍;機車;中和斷肢;大腿;卡車;送醫搶救
Using image system prompt key: stable_diffusion_prompt
📝 傳遞給 LLM 的 user_input (關鍵詞): Main character: kirby
新北市;中和區;車禍;機車;中和斷肢;大腿;卡車;送醫搶救


OpenRouter API 請求失敗 (嘗試 1/10): HTTP None - 404 Client Error: Not Found for url: https://openrouter.ai/api/v1/chat/completions。（當前模型: qwen/qwen3-235b-a22b:free）等待 4.4 秒後重試...
OpenRouter API 請求失敗 (嘗試 2/10): HTTP None - 429 Client Error: Too Many Requests for url: https://openrouter.ai/api/v1/chat/completions。（當前模型: minimax/minimax-m2:free）等待 6.4 秒後重試...
OpenRouter API 請求失敗 (嘗試 3/10): HTTP None - 404 Client Error: Not Found for url: https://openrouter.ai/api/v1/chat/completions。（當前模型: tngtech/deepseek-r1t2-chimera:free）等待 10.2 秒後重試...
2025-12-16 22:11:15,126 - examples.simple_content_service - INFO - 描述生成完成，共 1 個描述
2025-12-16 22:11:15,127 - examples.simple_content_service - INFO - 描述 1: "Kirby in shock amidst a chaotic urban traffic accident scene in Zhonghe District, New Taipei City, rain-slick asphalt reflecting red and blue emergency vehicle lights, overturned scooter with metal debris, paramedics in reflective uniforms attending to severed limb trauma near a delivery truck's rear whee

Image descriptions : ['"Kirby in shock amidst a chaotic urban traffic accident scene in Zhonghe District, New Taipei City, rain-slick asphalt reflecting red and blue emergency vehicle lights, overturned scooter with metal debris, paramedics in reflective uniforms attending to severed limb trauma near a delivery truck\'s rear wheels, volumetric light beams cutting through misty night air, Kirby\'s pink spherical body contrasting against gritty wet concrete textures, ambulance with flashing lights partially visible in background, low-angle dramatic composition emphasizing scale disparity between Kirby and industrial vehicle."']
生成描述花費 : 40.68 秒
工作流已提交，prompt_id: 2e0d3a48-22b6-4aaa-9d75-f1d5ed067395
開始等待工作流 2e0d3a48-22b6-4aaa-9d75-f1d5ed067395 完成...
  → 佇列中還有 1 個任務
  → 佇列中還有 1 個任務
  → 正在處理節點: 182
  → 正在處理節點: 260:255
  → 正在處理節點: 260:257
  → 正在處理節點: 260:258
  → 正在處理節點: 260:259
  → 正在處理節點: 272:264
  → 正在處理節點: 272:266
  → 正在處理節點: 272:267
  → 正在處理節點: 272:268
  → 正在處理節點: 269:264
  → 正在處理節點: 269